In [ ]:
import datetime as dt

import requests
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight
import xgboost as xgb


mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

tf.random.set_seed(42)

In [ ]:
def get_techninal_indicator_daily_time_series(symbol: str, indicator: str, series_type='close', time_period=7, interval='daily'):
    params = {
        'apikey': 'KNPL6J9N740SLRRG',
        'symbol': symbol,
        'function': indicator,
        'interval': interval
    }

    if indicator == 'AD':
        indicator = 'Chaikin A/D'

    if indicator in ['SMA', 'WMA', 'RSI', 'BBANDS', 'TRIX']:
        params['time_period'] = time_period
        params['series_type'] = series_type

    if indicator in ['DX', 'MFI', 'AROON', 'ADX']:
        params['time_period'] = time_period

    if indicator in ['MACD', 'PPO']:
        params['series_type'] = series_type

    json_response = requests.get(
        url='https://www.alphavantage.co/query',
        params=params
    ).json()
    time_series = []

    for date, data in json_response[f"Technical Analysis: {indicator}"].items():
        if indicator == 'STOCH':
            time_series.append(
            {
                "date": date,
                "SlowK": float(data["SlowK"]),
                "SlowD": float(data["SlowD"])
            }
        )
        elif indicator == 'AROON':
            time_series.append(
            {
                "date": date,
                "AroonDown": float(data["Aroon Down"]),
                "AroonUp": float(data["Aroon Up"])
            }
        )
        elif indicator == 'MACD':
            time_series.append(
            {
                "date": date,
                "MACD": float(data["MACD"]),
                "MACD_Signal": float(data["MACD_Signal"]),
                "MACD_Hist": float(data["MACD_Hist"]),
            }
        )
        elif indicator == 'BBANDS':
            time_series.append(
              {
                "date": date,
                "Real_Upper_Band": float(data["Real Upper Band"]),
                "Real_Lower_Band": float(data["Real Lower Band"])
              }
            )
        else:
          time_series.append(
              {
                  "date": date,
                  f"{indicator}": float(data[indicator])
              }
          )

    return pd.DataFrame(time_series)


def get_crypto_daily_time_series(symbol: str, market: str = 'EUR'):
    print(f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={market}&apikey=KNPL6J9N740SLRRG')
    json_response = requests.get(f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={market}&apikey=KNPL6J9N740SLRRG').json()
    print(json_response)
    time_series = []

    for date, data in json_response["Time Series (Digital Currency Daily)"].items():
        time_series.append(
          {
              "date": date,
              "open": float(data["1b. open (USD)"]),
              "high": float(data["2b. high (USD)"]),
              "low": float(data["3b. low (USD)"]),
              "close": float(data["4b. close (USD)"]),
              "volume": float(data["5. volume"]),
          }
        )

    return pd.DataFrame(time_series)


def get_dataset(symbol: str, indicators=None, time_period=7, interval='daily'):
    if symbol in ['BTC', 'ETH', 'SOL', 'ADA', 'AVAX', 'TRX', 'MATIC', 'LTC', 'UNI', 'ATOM', 'MKR', 'GRT', 'SNX', 'NEO', 'GNO', 'ALGO']:
        market = 'EUR'
    else:
        market = 'USD'

    time_series_df = get_crypto_daily_time_series(symbol, market)
    symbol = f"{symbol}{market}"

    indicators_dfs = []
    if indicators:
      for indicator in indicators:
          indicators_dfs.append(get_techninal_indicator_daily_time_series(symbol, indicator, time_period=time_period))

    merged_df = pd.merge(time_series_df, indicators_dfs[0], on='date')
    for df in indicators_dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='date')

    return merged_df


def create_prediction_input(symbol: str, indicators, time_period=7, interval='daily'):
    if symbol in ['BTC', 'ETH', 'SOL', 'ADA', 'AVAX', 'TRX', 'MATIC', 'LTC', 'UNI', 'ATOM', 'MKR', 'GRT', 'SNX', 'NEO', 'GNO', 'ALGO']:
        market = 'EUR'
    else:
        market = 'USD'

    time_series_df = get_crypto_daily_time_series(symbol, market)
    symbol = f"{symbol}{market}"

    indicators_dfs = []
    if indicators:
      for indicator in indicators:
          indicators_dfs.append(get_techninal_indicator_daily_time_series(symbol, indicator, time_period=time_period))

    merged_df = pd.merge(time_series_df, indicators_dfs[0], on='date')
    for df in indicators_dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='date')

    merged_df['date'] = pd.to_datetime(merged_df['date'])
    merged_df.sort_values(by='date', inplace=True)

    merged_df.reset_index(inplace=True, drop=True)
    prediction_input_df = merged_df.drop(['close', 'volume', 'open', 'high', 'low'], axis=1)
    prediction_input_df['OBV_pct_change'] = prediction_input_df['OBV'].pct_change() * 100
    prediction_input_df['AD_pct_change'] = prediction_input_df['Chaikin A/D'].pct_change() * 100
    prediction_input_df['TRIX'] = prediction_input_df['TRIX'] * 100
    prediction_input_df['BBANDS_distance_pct'] = ((prediction_input_df['Real_Upper_Band'] - prediction_input_df['Real_Lower_Band']) / prediction_input_df['Real_Lower_Band']) * 100
    prediction_input_df.drop(columns=['OBV', 'Chaikin A/D', 'Real_Upper_Band', 'Real_Lower_Band', 'date'], axis=1, inplace=True)
    prediction_input_df.dropna(inplace=True)
    return prediction_input_df.iloc[-1:]


In [ ]:
SYMBOL = 'AVAX'
INDICATORS=['OBV', 'AD', 'ADX', 'AROON', 'MACD', 'RSI', 'STOCH', 'MFI', 'DX', 'TRIX', 'BBANDS', 'PPO']

df = get_dataset(symbol=SYMBOL, indicators=INDICATORS)
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)

# # Find the latest date in the DataFrame
# latest_date = df['date'].max()
# # Calculate the date one year ago from the latest date
# one_year_ago = latest_date - pd.DateOffset(years=1)
# # Filter the DataFrame to keep only values from the last year
# df = df[df['date'] >= one_year_ago]

df.reset_index(inplace=True, drop=True)
dataset = df.drop(['open', 'high', 'low'], axis=1)

https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=AVAX&market=EUR&apikey=KNPL6J9N740SLRRG
{'Meta Data': {'1. Information': 'Daily Prices and Volumes for Digital Currency', '2. Digital Currency Code': 'AVAX', '3. Digital Currency Name': 'Avalanche', '4. Market Code': 'EUR', '5. Market Name': 'Euro', '6. Last Refreshed': '2024-04-07 00:00:00', '7. Time Zone': 'UTC'}, 'Time Series (Digital Currency Daily)': {'2024-04-07': {'1a. open (EUR)': '44.45045600', '1b. open (USD)': '48.19000000', '2a. high (EUR)': '45.27139200', '2b. high (USD)': '49.08000000', '3a. low (EUR)': '44.37666400', '3b. low (USD)': '48.11000000', '4a. close (EUR)': '45.24372000', '4b. close (USD)': '49.05000000', '5. volume': '207000.63000000', '6. market cap (USD)': '207000.63000000'}, '2024-04-06': {'1a. open (EUR)': '41.78472000', '1b. open (USD)': '45.30000000', '2a. high (EUR)': '45.17915200', '2b. high (USD)': '48.98000000', '3a. low (EUR)': '41.55412000', '3b. low (USD)': '45.05000000', '4a

In [ ]:
dataset.head()

,date,close,volume,OBV,Chaikin A/D,ADX,AroonDown,AroonUp,MACD,MACD_Signal,MACD_Hist,RSI,SlowK,SlowD,MFI,DX,TRIX,Real_Upper_Band,Real_Lower_Band,PPO
0,2021-08-15,18.762,1.404169e+06,22654653.0,7.940130e+06,62.1490,14.2857,100.0,1.7628,1.4430,0.3198,85.6248,93.0812,84.9694,86.7186,58.9311,2.5148,17.6510,14.1686,16.1044
1,2021-08-16,19.071,3.800909e+06,26455561.0,5.920235e+06,64.2750,0.0000,100.0,1.8358,1.5216,0.3143,86.7609,83.2836,86.6587,88.5425,77.0311,2.5573,18.1619,14.4225,16.5387
2,2021-08-17,23.446,1.017204e+07,36627597.0,1.027390e+07,67.7202,28.5714,100.0,2.1941,1.6561,0.5380,94.2576,79.6555,85.3401,93.5986,88.3915,2.9781,21.1647,13.2714,18.1396
3,2021-08-18,29.993,1.287246e+07,49500061.0,2.071137e+07,71.3691,14.2857,100.0,2.9315,1.9112,1.0204,97.1123,78.4099,80.4497,96.3047,93.2625,4.0497,26.9556,10.9869,20.7904
4,2021-08-19,30.653,1.188454e+07,61384600.0,1.796951e+07,74.6947,0.0000,100.0,3.5244,2.2338,1.2906,97.2718,84.2560,80.7738,100.0000,94.6484,5.1116,30.4019,11.2508,22.8183


In [ ]:
dataset['OBV_pct_change'] = dataset['OBV'].pct_change() * 100
dataset['AD_pct_change'] = dataset['Chaikin A/D'].pct_change() * 100
dataset['TRIX'] = dataset['TRIX'] * 100
dataset['BBANDS_distance_pct'] = ((dataset['Real_Upper_Band'] - dataset['Real_Lower_Band']) / df['Real_Lower_Band']) * 100

# Create target variable
look_ahead_days = 7
dataset['max_next_7_days'] = dataset['close'].rolling(window=look_ahead_days).max().shift(-look_ahead_days + 1)
dataset.dropna(inplace=True)

# Create target column
percentage_difference = (dataset['max_next_7_days'] - dataset['close']) / dataset['close']
dataset['target'] = (percentage_difference >= 0.05).astype(int)
# dataset['target'] = (dataset['max_next_7_days'] > dataset['close']).astype(int)

dataset.drop(columns=['close', 'volume', 'OBV', 'Chaikin A/D', 'max_next_7_days', 'Real_Upper_Band', 'Real_Lower_Band'], axis=1, inplace=True)
dataset.dropna(inplace=True)
dataset.head()

,date,ADX,AroonDown,AroonUp,MACD,MACD_Signal,MACD_Hist,RSI,SlowK,SlowD,MFI,DX,TRIX,PPO,OBV_pct_change,AD_pct_change,BBANDS_distance_pct,target
1,2021-08-16,64.2750,0.0000,100.0,1.8358,1.5216,0.3143,86.7609,83.2836,86.6587,88.5425,77.0311,255.73,16.5387,16.777604,-25.439066,25.927544,1
2,2021-08-17,67.7202,28.5714,100.0,2.1941,1.6561,0.5380,94.2576,79.6555,85.3401,93.5986,88.3915,297.81,18.1396,38.449519,73.538733,59.476016,1
3,2021-08-18,71.3691,14.2857,100.0,2.9315,1.9112,1.0204,97.1123,78.4099,80.4497,96.3047,93.2625,404.97,20.7904,35.144167,101.592029,145.343090,1
4,2021-08-19,74.6947,0.0000,100.0,3.5244,2.2338,1.2906,97.2718,84.2560,80.7738,100.0000,94.6484,511.16,22.8183,24.009140,-13.238434,170.219895,1
5,2021-08-20,77.7853,0.0000,100.0,4.5744,2.7019,1.8725,98.5093,91.5384,84.7348,100.0000,96.3288,657.96,26.9240,13.434725,45.162143,270.485433,1


In [ ]:
dataset.drop(['date'], inplace=True, axis=1)

n = len(dataset)
train_dataset = dataset[0:int(n*0.97)]
# validation_dataset = dataset[int(n*0.97):int(n*0.98)]
test_dataset = dataset[int(n*0.97):]

In [ ]:
X_train = train_dataset.drop(columns=['target'], axis=1)
y_train = train_dataset['target']

X_test = test_dataset.drop(columns=['target'], axis=1)
y_test = test_dataset['target']

# X_val = validation_dataset.drop(columns=['target'], axis=1)
# y_val = validation_dataset['target']

In [ ]:
y_train.value_counts()

target
0    492
1    439
Name: count, dtype: int64

In [ ]:
y_test.value_counts()

target
1    15
0    14
Name: count, dtype: int64

In [ ]:
classes = np.unique(y_train)
weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

print("Class weights:", class_weights)

classifiers = {
    "Random Forest": RandomForestClassifier(class_weight=class_weights),
    "Support Vector Machine": SVC(probability=True, class_weight=class_weights),
    "XGBoost": xgb.XGBClassifier(scale_pos_weight=y_train.value_counts()[0] / y_train.value_counts()[1]),
    "HistGradientBoostingClassifier": HistGradientBoostingClassifier(class_weight=class_weights),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "RidgeClassifier": RidgeClassifier(class_weight=class_weights),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "MLPClassifier": MLPClassifier()
}


def get_overall_score(accuracy: float, precision: float, negative_accuracy: float, positive_accuracy: float) -> float:
    return (0.3 * accuracy) + (0.3 * precision) + (0.1 * negative_accuracy) + (0.3 * positive_accuracy)


def evaluate_classifier(classifier, X_train, y_train, X_test, y_test, threshold: float = 0.5):
    classifier.fit(X_train, y_train)
    labels =  classifier.classes_
    if labels[0] != 0:
        raise Exception("LABELS ARE FUCKED UP")

    if not isinstance(classifier, RidgeClassifier):
      y_prob = classifier.predict_proba(X_test)
      y_pred = [
          1 if prediction_prob[1] > threshold else 0
          for prediction_prob in y_prob
      ]
    else:
      y_pred = classifier.predict(X_test)

    cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    negative_accuracy = cm[0][0] / (cm[0][0] + cm[0][1])
    positive_accuracy = cm[1][1] / (cm[1][0] + cm[1][1])
    return {
        "accuracy": accuracy,
        "precision": precision,
        "positive_accuracy": positive_accuracy,
        "negative_accuracy": negative_accuracy,
        "cm": cm,
        "overall_score": get_overall_score(accuracy, precision, negative_accuracy, positive_accuracy)
    }


# Initialize lists to store results
results = {
    'Classifier': [],
    'Accuracy': [],
    'Precision': [],
    'Positive_Accuracy': [],
    'Negative_Accuracy': [],
    'Confusion Matrix': [],
    'Overall Score': []
  }

# Iterate through classifiers
for clf_name, clf in classifiers.items():
    # Evaluate classifier
    metrics = evaluate_classifier(clf, X_train, y_train, X_test, y_test)

    # Store results
    results['Classifier'].append(clf_name)
    results['Accuracy'].append(metrics['accuracy'])
    results['Precision'].append(metrics['precision'])
    results['Positive_Accuracy'].append(metrics['positive_accuracy'])
    results['Negative_Accuracy'].append(metrics['negative_accuracy'])
    results['Confusion Matrix'].append(metrics['cm'])
    results['Overall Score'].append(metrics['overall_score'])

Class weights: {0: 0.9461382113821138, 1: 1.060364464692483}


In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fit the model
model.fit(X_train_scaled, y_train, epochs=100, class_weight=class_weights)

# Evaluate the model
y_pred_probs = model.predict(X_test_scaled)
print(y_pred_probs)
# Convert probabilities to binary predictions
# threshold = y_train.value_counts()[1] / (y_train.value_counts()[0] + y_train.value_counts()[1] )
y_pred = (y_pred_probs > 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred, labels=[0,1])
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
negative_accuracy = cm[0][0] / (cm[0][0] + cm[0][1])
positive_accuracy = cm[1][1] / (cm[1][0] + cm[1][1])
overall_score = get_overall_score(accuracy, precision, negative_accuracy, positive_accuracy)

classifiers['Neural Net'] = model

results['Classifier'].append('Neural Net')
results['Accuracy'].append(accuracy)
results['Precision'].append(precision)
results['Positive_Accuracy'].append(positive_accuracy)
results['Negative_Accuracy'].append(negative_accuracy)
results['Confusion Matrix'].append(cm)
results['Overall Score'].append(overall_score)

Epoch 1/100
30/30 [==============================] - 2s 2ms/step - loss: 0.6823 - accuracy: 0.5682
Epoch 2/100
30/30 [==============================] - 0s 2ms/step - loss: 0.6488 - accuracy: 0.6122
Epoch 3/100
30/30 [==============================] - 0s 2ms/step - loss: 0.6336 - accuracy: 0.6251
Epoch 4/100
30/30 [==============================] - 0s 2ms/step - loss: 0.6252 - accuracy: 0.6488
Epoch 5/100
30/30 [==============================] - 0s 2ms/step - loss: 0.6169 - accuracy: 0.6434
Epoch 6/100
30/30 [==============================] - 0s 2ms/step - loss: 0.6080 - accuracy: 0.6713
Epoch 7/100
30/30 [==============================] - 0s 2ms/step - loss: 0.5985 - accuracy: 0.6799
Epoch 8/100
30/30 [==============================] - 0s 2ms/step - loss: 0.5909 - accuracy: 0.6713
Epoch 9/100
30/30 [==============================] - 0s 2ms/step - loss: 0.5845 - accuracy: 0.6670
Epoch 10/100
30/30 [==============================] - 0s 2ms/step - loss: 0.5773 - accuracy: 0.6907
Epoch 11/

In [ ]:
results['Classifier'].append('Naive')
naive_positive_accuracy = y_test.value_counts()[1] / (y_test.value_counts()[0] + y_test.value_counts()[1] )
naive_negative_accuracy = y_test.value_counts()[0] / (y_test.value_counts()[0] + y_test.value_counts()[1] )
results['Accuracy'].append(naive_positive_accuracy)
results['Precision'].append(None)
results['Positive_Accuracy'].append(naive_positive_accuracy)
results['Negative_Accuracy'].append(naive_negative_accuracy)
results['Confusion Matrix'].append(None)
results['Overall Score'].append(0)

results_df = pd.DataFrame(results)
results_df.sort_values(by=['Overall Score'], ascending=False, inplace=True)
results_df.reset_index(inplace=True)
results_df

,index,Classifier,Accuracy,Precision,Positive_Accuracy,Negative_Accuracy,Confusion Matrix,Overall Score
0,3,HistGradientBoostingClassifier,0.827586,0.857143,0.800000,0.857143,"[[12, 2], [3, 12]]",0.831133
1,0,Random Forest,0.793103,0.846154,0.733333,0.857143,"[[12, 2], [4, 11]]",0.797491
2,2,XGBoost,0.758621,0.750000,0.800000,0.714286,"[[10, 4], [3, 12]]",0.764015
3,7,MLPClassifier,0.551724,0.535714,1.000000,0.071429,"[[1, 13], [0, 15]]",0.633374
4,4,AdaBoostClassifier,0.586207,0.714286,0.333333,0.857143,"[[12, 2], [10, 5]]",0.575862
5,6,KNeighborsClassifier,0.517241,0.526316,0.666667,0.357143,"[[5, 9], [5, 10]]",0.548781
6,8,Neural Net,0.517241,0.545455,0.400000,0.642857,"[[9, 5], [9, 6]]",0.503094
7,1,Support Vector Machine,0.517241,0.571429,0.266667,0.785714,"[[11, 3], [11, 4]]",0.485172
8,5,RidgeClassifier,0.482759,0.500000,0.400000,0.571429,"[[8, 6], [9, 6]]",0.471970
9,9,Naive,0.517241,NaN,0.517241,0.482759,None,0.000000


### The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

In [ ]:
# Get best performing model
top_classifier_name = results_df['Classifier'][0]
classifier = classifiers[top_classifier_name]

# Get prediction
prediction_input = create_prediction_input(symbol=SYMBOL, indicators=INDICATORS)
if top_classifier_name in ['RidgeClassifier', 'Neural Net']:
    if top_classifier_name == 'Neural Net':
        prediction = classifier.predict(scaler.transform(prediction_input))
    else:
        prediction = classifier.predict(prediction_input)
else:
    prediction = classifier.predict_proba(prediction_input)

prediction

https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=AVAX&market=EUR&apikey=KNPL6J9N740SLRRG
{'Meta Data': {'1. Information': 'Daily Prices and Volumes for Digital Currency', '2. Digital Currency Code': 'AVAX', '3. Digital Currency Name': 'Avalanche', '4. Market Code': 'EUR', '5. Market Name': 'Euro', '6. Last Refreshed': '2024-04-07 00:00:00', '7. Time Zone': 'UTC'}, 'Time Series (Digital Currency Daily)': {'2024-04-07': {'1a. open (EUR)': '44.45045600', '1b. open (USD)': '48.19000000', '2a. high (EUR)': '45.27139200', '2b. high (USD)': '49.08000000', '3a. low (EUR)': '44.37666400', '3b. low (USD)': '48.11000000', '4a. close (EUR)': '45.24372000', '4b. close (USD)': '49.05000000', '5. volume': '207000.63000000', '6. market cap (USD)': '207000.63000000'}, '2024-04-06': {'1a. open (EUR)': '41.78472000', '1b. open (USD)': '45.30000000', '2a. high (EUR)': '45.17915200', '2b. high (USD)': '48.98000000', '3a. low (EUR)': '41.55412000', '3b. low (USD)': '45.05000000', '4a

array([[0.90319606, 0.09680394]])